In [1]:
import pandas as pd  # pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##  XGBoost
https://www.kaggle.com/tilii7/xgboost-simple-starter-more-features

In [2]:
#Importing Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import log_loss
from operator import itemgetter
from sklearn import svm
from sklearn.preprocessing import StandardScaler


/var/local/conda/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
df_train = pd.read_csv("../datasets/train1_50labels_withdaytimes_app60000.csv", sep=",")
#df_train = df_train.set_index("device_id")
#df_train.drop(columns=["Unnamed: 0"], inplace=True)
df_train.head(1)

,Unnamed: 0,device_id,events_num,min_timestamp,max_timestamp,latitude_events_median,longitude_events_median,events_per_day,label_548,label_i_548,...,app_9136721329817567486,app_-2145434552412465677,app_6868430133575209713,app_5824972015480403938,gender,age,group,phone_brand,device_model,phone_brand_eng
0,0,-8076087639492063270,0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,M,35,M32-38,小米,MI 2,Xiaomi


In [4]:
## TODO : nan, infiniti -> 0
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)

In [5]:
# factorize
if True:
    factor = pd.factorize(df_train['phone_brand_eng'])
    df_train.phone_brand_eng = factor[0]
    definitions_phone_brand_eng = factor[1]
    factor = pd.factorize(df_train['device_model'])
    df_train.device_model = factor[0]
    definitions_device_model = factor[1]
    factor = pd.factorize(df_train['group'])
    df_train.group = factor[0]
    definitions = factor[1]

In [6]:
df_train_events=df_train[df_train["events_num"] > 0]
df_train_noevents=df_train[df_train["events_num"] == 0][["phone_brand_eng", "device_model", "group"]]
df_train_noevents.head(1)

,phone_brand_eng,device_model,group
0,0,0,0


In [7]:
def doSVM(df_train, magyarazo_valtozok):

    # split
    X_train, X_test, y_train, y_test = train_test_split(df_train[magyarazo_valtozok], df_train['group'], test_size=0.1,random_state=109) # 90% training and 10% test
        
    scaler = StandardScaler().fit(X_train)  
    X_train_scd = pd.DataFrame(scaler.transform(X_train))
    X_train_scd.columns = X_train.columns
    X_test_scd = pd.DataFrame(scaler.transform(X_test))
    X_test_scd.columns = X_test.columns
    X_test_scd.head()
        #Create a svm Classifier
    clf = svm.SVC(kernel='linear', probability=True) # Linear Kernel rbf/linear
    #Train the model using the training sets
    clf.fit(X_train_scd, y_train)
    #Predict the response for test dataset
    y_pred = clf.predict_proba(X_test_scd)
    
    #imp,names = zip(*sorted(zip(clf.coef_,magyarazo_valtozok)))
    #print(clf.coef_)
    #print(magyarazo_valtozok)
    #feature_importances = pd.DataFrame((magyarazo_valtozok, clf.coef_),
    #                               index = X_train.columns,
    #                                columns=['importance']).sort_values('importance', ascending=False)
    score = log_loss(y_test.tolist(), y_pred)
    p = metrics.accuracy_score(y_test, np.argmax(y_pred, axis=1))
    #p = metrics.accuracy_score(y_test, y_pred)


    return clf.coef_, score, p
    

    

In [8]:
results = {}


In [9]:
magyarazo_valtozok_app_label = []
for m in df_train.columns:
    if m not in ['Unnamed: 0', 'device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        magyarazo_valtozok_app_label += [m]
        
magyarazo_valtozok_app = []
for m in df_train.columns:
    if m not in ['Unnamed: 0', 'device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        if m.startswith("label_"):
            continue
        magyarazo_valtozok_app += [m]

magyarazo_valtozok_label = []
for m in df_train.columns:
    if m not in ['Unnamed: 0', 'device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        if m.startswith("app_"):
            continue
        magyarazo_valtozok_label += [m]

magyarazo_valtozok_nolabel_noapp = []
for m in df_train.columns:
    if m not in ['Unnamed: 0', 'device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        if m.startswith("app_"):
            continue
        if m.startswith("label_"):
            continue            
        magyarazo_valtozok_nolabel_noapp += [m]

        
        
magyarazo_valtozok_no_events = []
for m in df_train_noevents.columns:
    if m not in ['Unnamed: 0', 'device_id', 'min_timestamp', 'max_timestamp','gender', 'age', 'group', 'phone_brand' ]:
        magyarazo_valtozok_no_events += [m]



In [ ]:
results={}
results["NoEvent, Event - Label:yes, App:yes"] = (doSVM(df_train, magyarazo_valtozok_app_label))
results["NoEvent, Event - Label:no, App:yes"] = (doSVM(df_train, magyarazo_valtozok_app))
results["NoEvent, Event - Label:yes, App:no"] = (doSVM(df_train, magyarazo_valtozok_label))
results["NoEvent, Event - Label:no, App:no"] = (doSVM(df_train, magyarazo_valtozok_nolabel_noapp))
results["Event - Label:yes, App:yes"] = (doSVM(df_train_events, magyarazo_valtozok_app_label))
results["Event - Label:no, App:yes"] = (doSVM(df_train_events, magyarazo_valtozok_app))
results["Event - Label:yes, App:no"] = (doSVM(df_train_events, magyarazo_valtozok_label))
results["Event - Label:no, App:no"] = (doSVM(df_train_events, magyarazo_valtozok_nolabel_noapp))
results["NoEvent"] = (doSVM(df_train_noevents, magyarazo_valtozok_no_events))


In [ ]:

for i in results:
    print("%s - Log loss: %f, Accuary:%f"%(i, results[i][1], results[i][2]))
    continue
    importance = results[i][0][:30]
    # Plot the feature importances of the forest
    df1 = pd.DataFrame(importance)
    df1.rename(columns={0:"feature", 1:"cou"}, inplace=True)
    df1=df1.set_index("feature")
    df1.plot.bar(title="%s - Log loss: %f, Accuary:%f"%(i,results[i][1], results[i][2]), figsize=(14, 7))